# Task 2 · Paso 1 — Split 70/15/15 en AG News
Verificamos tamaños (train/validation/test) y equilibrio de etiquetas.

In [ ]:
# !pip -q install -r ../requirements.txt
from src.train_eval import set_seed, stratified_split_agnews
set_seed(42)
splits = stratified_split_agnews(train=0.70, val=0.15, test=0.15)
{k: len(splits[k]) for k in ["train","validation","test"]}

In [ ]:
# Revisión rápida de equilibrio de etiquetas
import pandas as pd, collections
def label_dist(ds):
    return pd.Series(collections.Counter(list(ds["label"]))).sort_index()
{k: label_dist(splits[k]) for k in ["train","validation","test"]}